In [ ]:
# Python3 

import googleapiclient, googleapiclient.discovery
import httplib2
from oauth2client import file, client, tools

SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('storage.json')
creds = store.get()
if not creds or creds.invalid:
    print ("heere")
    flow = client.flow_from_clientsecrets('gdrive_Oauth.json', SCOPES)
    creds = tools.run_flow(flow, store)
DRIVE = googleapiclient.discovery.build('drive', 'v3', http=creds.authorize(httplib2.Http()))
# A drive service 

In [ ]:
def list_folders(DRIVE):
    files = DRIVE.files().list().execute().get('files', [])
    for f in files:
        if f['mimeType'] == 'application/vnd.google-apps.folder':
            print(f['name'], f['id'], f['mimeType'])
#        print (f)
            
    return None

list_folders(DRIVE)

In [ ]:
def search_file_by_name(DRIVE, name):
    files = DRIVE.files().list(q="name contains '" + name + "'").execute().get('files', [])
    for f in files:
        print (f)
        
search_file_by_name(DRIVE, 'QA')

In [ ]:
def get_file_metadata(DRIVE, fileid, fields=None):
    metadata = {}
    
    if fields == None:
        file = DRIVE.files().get(fileId=fileid, fields="*").execute()
        metadata = file
        return metadata
    
    for field in fields:
        file = DRIVE.files().get(fileId=fileid, fields=field).execute()
        metadata[field] = file.get(field, '')
        
    return metadata
    
get_file_metadata(DRIVE, '', ['name', 'parents'])


In [ ]:
def upload_to_folder(DRIVE, file_info, GDrive_folder_ID):    
    file_metadata =  {
        'name': file_info['cloud_name'],
        'parents': [GDrive_folder_ID]
    }
    
    media = googleapiclient.http.MediaFileUpload(file_info['local_path'],
                                            mimetype=file_info.get('type', None),
                                            resumable=True)
    file = DRIVE.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()
    file_ID= file.get('id', '')
    
    print ('File ID: %s' % file.get('id')    )
    
    GDrive_metadata = DRIVE.files().get(fileId=file_ID, fields="*").execute()
    
    for key,value in GDrive_metadata.iteritems():
        print (key, value)

file_info = {'local_path':'QA_test.json', 
              'type':'application/json', 
              'cloud_name':'QA_test_data.json'}
upload_to_folder(DRIVE, file_info, '')